### Classification Using Artificial Neural Networks from Sci-kit learn

Use the Scikit-Learn MLP Classifier.
Train your dataset.
Get accuracy scores and confusion matrix.
You need a minimum accuracy score of 94%

Build another neural network using Keras.
Re-train your dataset.
Get accuracy scores and confusion matrix.
Compare results with the outputs of the classifiers in through project 2.


Split your data as follows:
80% training set
10% validation set
10% test set

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import optuna

from load_preprocessed_bcd import load_preprocessed_bcd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report



# Load pre-processed breast cancer dataset using the wrapper function load_preprocessed_bcd()
### The wrapper function returns four different dataframes. 

### All dataframes share these characteristics:
- Data belongs to the breast cancer dataset.
- Data is free of Null values.
- Data is standardized using the Z-score.

### Each dataframe differs by the dimensionality reduction used on it:
- df_scaled: all features are kept
- df_hypo: the kept features are those that show a significant difference (Mann Whitney-U test, p<0.001) between the groups Benign and Malign.
- df_pca: tHe kept features are the prinicpal components that account for 95% of the data's variance.
- df_correl: the kept features show a correlation with the target higher than 0.2, and a correlation with each other below 0.7

In [2]:
%%capture
df_scaled, df_hypo, df_pca, df_correl = load_preprocessed_bcd()

# 1. Use all features to discern between benign and malign breast measurements

In [3]:
X = df_scaled.drop(columns=['diagnosis'])
y = np.round(df_scaled['diagnosis']=='M').astype(int)
print("Nr. Features: ", X.shape[1])

Nr. Features:  30


In [4]:
display(X.head(5))
display(y.head(5))

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,...,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,...,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100


0    1
1    1
2    1
3    1
4    1
Name: diagnosis, dtype: int32

In [5]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.25, random_state=0)

In [6]:
print("Nr. test events: ", X_test.shape[0])
print("Nr. validation events: ", X_valid.shape[0])
print("Nr. train events: ", X_train.shape[0])

Nr. test events:  143
Nr. validation events:  107
Nr. train events:  319


#### Define the objective function to use in the hyperparameter optimization with Optuna

In [7]:
def objective_ann(trial, X_train, X_valid, y_train, y_valid):
    
    params = {
        'n_hidden_layers': trial.suggest_int('n_hidden_layers', 1, 10, step=1),
        'nodes_per_layer': trial.suggest_int('nodes_per_layer', 10, 100, step=10),
        'activation': trial.suggest_categorical('activation', ['identity', 'logistic', 'tanh', 'relu']),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam']),
        'learning_rate': trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive']),
        "max_iter": trial.suggest_categorical("max_iter", [1000]),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "early_stopping": trial.suggest_categorical("early_stopping", [True]),
    }

    layers_ls = []
    n_layers = params['n_hidden_layers']
    nodes_per_layer = params['nodes_per_layer']
    for i in range(n_layers):
        layers_ls.append(nodes_per_layer)


    model = MLPClassifier(
        hidden_layer_sizes=layers_ls,
        activation=params['activation'],
        solver=params['solver'],
        learning_rate=params['learning_rate'],
        max_iter=params['max_iter'],
        random_state=params['random_state'],
        early_stopping=params['early_stopping'],
    )

    model.fit(X_train, y_train)
    y_predicted = model.predict(X_valid)
    (tn, fp, fn, tp) = confusion_matrix(y_valid, y_predicted).ravel()
    accuracy = float(tp+tn)/float(tn+fp+fn+tp)
    f1score = float(2*tp)/float(2*tp+fp+fn)
    mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))
    
    return mcc

In [ ]:
%%capture
optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.set_verbosity(optuna.logging.WARNING)

study_ann = optuna.create_study(direction = "maximize")
func = lambda trial: objective_ann(trial, X_train, X_valid, y_train, y_valid)
study_ann.optimize(func, n_trials = 100, timeout=600)

In [9]:
best_params = study_ann.best_trial.params
layers_ls = []
n_layers = best_params['n_hidden_layers']
nodes_per_layer = best_params['nodes_per_layer']
for i in range(n_layers):
    layers_ls.append(nodes_per_layer)

model = MLPClassifier(
    hidden_layer_sizes=layers_ls,
    activation=best_params['activation'],
    solver=best_params['solver'],
    learning_rate=best_params['learning_rate'],
    max_iter=best_params['max_iter'],
    random_state=best_params['random_state'],
    early_stopping=best_params['early_stopping'],
)

model.fit(X_train, y_train)

MLPClassifier(early_stopping=True, hidden_layer_sizes=[70, 70, 70, 70, 70],
              max_iter=1000, random_state=42, solver='lbfgs')

In [10]:
print("Best Score Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key,value))

Best Score Params: 
    n_hidden_layers: 5
    nodes_per_layer: 70
    activation: relu
    solver: lbfgs
    learning_rate: constant
    max_iter: 1000
    random_state: 42
    early_stopping: True


### Run the ANN on the test set

In [11]:
y_predicted_train = model.predict(X_valid)
(tn, fp, fn, tp) = confusion_matrix(y_valid, y_predicted_train).ravel()
accuracy = float(tp+tn)/float(tn+fp+fn+tp)
f1score = float(2*tp)/float(2*tp+fp+fn)
train_mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))

y_predicted_test = model.predict(X_test)
(tn, fp, fn, tp) = confusion_matrix(y_test, y_predicted_test).ravel()
accuracy = float(tp+tn)/float(tn+fp+fn+tp)
f1score = float(2*tp)/float(2*tp+fp+fn)
test_mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))

display(y_predicted_test)
print("Training MCC =", train_mcc)
print("Test MCC =", test_mcc)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1])

Training MCC = 0.9420771376946219
Test MCC = 0.925445136208044


# 2. Use df_hypo features

In [12]:
X = df_hypo.drop(columns=['diagnosis'])
y = np.round(df_hypo['diagnosis']=='M').astype(int)
print("Nr. Features: ", X.shape[1])

Nr. Features:  26


In [13]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.25, random_state=0)

In [ ]:
%%capture
optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.set_verbosity(optuna.logging.WARNING)

study_ann = optuna.create_study(direction = "maximize")
func = lambda trial: objective_ann(trial, X_train, X_valid, y_train, y_valid)
study_ann.optimize(func, n_trials = 100, timeout=600)

In [15]:
best_params = study_ann.best_trial.params
layers_ls = []
n_layers = best_params['n_hidden_layers']
nodes_per_layer = best_params['nodes_per_layer']
for i in range(n_layers):
    layers_ls.append(nodes_per_layer)

model = MLPClassifier(
    hidden_layer_sizes=layers_ls,
    activation=best_params['activation'],
    solver=best_params['solver'],
    learning_rate=best_params['learning_rate'],
    max_iter=best_params['max_iter'],
    random_state=best_params['random_state'],
    early_stopping=best_params['early_stopping'],
)

model.fit(X_train, y_train)

MLPClassifier(activation='tanh', early_stopping=True,
              hidden_layer_sizes=[80, 80, 80, 80, 80],
              learning_rate='invscaling', max_iter=1000, random_state=42,
              solver='lbfgs')

In [16]:
print("Best Score Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key,value))

Best Score Params: 
    n_hidden_layers: 5
    nodes_per_layer: 80
    activation: tanh
    solver: lbfgs
    learning_rate: invscaling
    max_iter: 1000
    random_state: 42
    early_stopping: True


### Run the ANN on the test set

In [17]:
y_predicted_train = model.predict(X_valid)
(tn, fp, fn, tp) = confusion_matrix(y_valid, y_predicted_train).ravel()
accuracy = float(tp+tn)/float(tn+fp+fn+tp)
f1score = float(2*tp)/float(2*tp+fp+fn)
train_mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))

y_predicted_test = model.predict(X_test)
(tn, fp, fn, tp) = confusion_matrix(y_test, y_predicted_test).ravel()
accuracy = float(tp+tn)/float(tn+fp+fn+tp)
f1score = float(2*tp)/float(2*tp+fp+fn)
test_mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))

display(y_predicted_test)
print("Training MCC =", train_mcc)
print("Test MCC =", test_mcc)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1])

Training MCC = 0.9806718882557136
Test MCC = 0.8955785926563062


# 3. Use df_pca features

In [18]:
X = df_pca.drop(columns=['diagnosis'])
y = np.round(df_pca['diagnosis']=='M').astype(int)
print("Nr. Features: ", X.shape[1])

Nr. Features:  10


In [19]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.25, random_state=0)

In [ ]:
%%capture
optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.set_verbosity(optuna.logging.WARNING)

study_ann = optuna.create_study(direction = "maximize")
func = lambda trial: objective_ann(trial, X_train, X_valid, y_train, y_valid)
study_ann.optimize(func, n_trials = 100, timeout=600)

In [21]:
best_params = study_ann.best_trial.params
layers_ls = []
n_layers = best_params['n_hidden_layers']
nodes_per_layer = best_params['nodes_per_layer']
for i in range(n_layers):
    layers_ls.append(nodes_per_layer)

model = MLPClassifier(
    hidden_layer_sizes=layers_ls,
    activation=best_params['activation'],
    solver=best_params['solver'],
    learning_rate=best_params['learning_rate'],
    max_iter=best_params['max_iter'],
    random_state=best_params['random_state'],
    early_stopping=best_params['early_stopping'],
)

model.fit(X_train, y_train)

MLPClassifier(activation='identity', early_stopping=True,
              hidden_layer_sizes=[70, 70, 70, 70, 70, 70],
              learning_rate='adaptive', max_iter=1000, random_state=42,
              solver='sgd')

In [22]:
print("Best Score Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key,value))

Best Score Params: 
    n_hidden_layers: 6
    nodes_per_layer: 70
    activation: identity
    solver: sgd
    learning_rate: adaptive
    max_iter: 1000
    random_state: 42
    early_stopping: True


### Run the ANN on the test set

In [23]:
y_predicted_train = model.predict(X_valid)
(tn, fp, fn, tp) = confusion_matrix(y_valid, y_predicted_train).ravel()
accuracy = float(tp+tn)/float(tn+fp+fn+tp)
f1score = float(2*tp)/float(2*tp+fp+fn)
train_mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))

y_predicted_test = model.predict(X_test)
(tn, fp, fn, tp) = confusion_matrix(y_test, y_predicted_test).ravel()
accuracy = float(tp+tn)/float(tn+fp+fn+tp)
f1score = float(2*tp)/float(2*tp+fp+fn)
test_mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))

display(y_predicted_test)
print("Training MCC =", train_mcc)
print("Test MCC =", test_mcc)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])

Training MCC = 0.9223486249227962
Test MCC = 0.8345678187810464


# 4. Use df_correl features

In [24]:
X = df_correl.drop(columns=['diagnosis'])
y = np.round(df_correl['diagnosis']=='M').astype(int)
print("Nr. Features: ", X.shape[1])

Nr. Features:  8


In [25]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.25, random_state=0)

In [ ]:
%%capture
optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.set_verbosity(optuna.logging.WARNING)

study_ann = optuna.create_study(direction = "maximize")
func = lambda trial: objective_ann(trial, X_train, X_valid, y_train, y_valid)
study_ann.optimize(func, n_trials = 100, timeout=600)

In [27]:
best_params = study_ann.best_trial.params
layers_ls = []
n_layers = best_params['n_hidden_layers']
nodes_per_layer = best_params['nodes_per_layer']
for i in range(n_layers):
    layers_ls.append(nodes_per_layer)

model = MLPClassifier(
    hidden_layer_sizes=layers_ls,
    activation=best_params['activation'],
    solver=best_params['solver'],
    learning_rate=best_params['learning_rate'],
    max_iter=best_params['max_iter'],
    random_state=best_params['random_state'],
    early_stopping=best_params['early_stopping'],
)

model.fit(X_train, y_train)

MLPClassifier(activation='tanh', early_stopping=True,
              hidden_layer_sizes=[90, 90, 90, 90], learning_rate='invscaling',
              max_iter=1000, random_state=42)

In [28]:
print("Best Score Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key,value))

Best Score Params: 
    n_hidden_layers: 4
    nodes_per_layer: 90
    activation: tanh
    solver: adam
    learning_rate: invscaling
    max_iter: 1000
    random_state: 42
    early_stopping: True


### Run the ANN on the test set

In [29]:
y_predicted_train = model.predict(X_valid)
(tn, fp, fn, tp) = confusion_matrix(y_valid, y_predicted_train).ravel()
accuracy = float(tp+tn)/float(tn+fp+fn+tp)
f1score = float(2*tp)/float(2*tp+fp+fn)
train_mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))

y_predicted_test = model.predict(X_test)
(tn, fp, fn, tp) = confusion_matrix(y_test, y_predicted_test).ravel()
accuracy = float(tp+tn)/float(tn+fp+fn+tp)
f1score = float(2*tp)/float(2*tp+fp+fn)
test_mcc = (float(tp*tn)-float(fp*fn))/np.sqrt(float(tp+fp)*float(tp+fn)*float(tn+fp)*float(tn+fn))

display(y_predicted_test)
print("Training MCC =", train_mcc)
print("Test MCC =", test_mcc)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])

Training MCC = 0.903245770030367
Test MCC = 0.9251281457541922
